In [4]:
! pip install timm kagglehub

In [2]:
pip install tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 299.7 kB/s eta 0:00:00 kB/s eta 0:00:01:06
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 381.9 kB/s eta 0:00:001m379.1 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import kagglehub
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")
print("Path to dataset files:", path)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████████████████████████████████████████████████| 105M/105M [00:12<00:00, 8.88MB/s]

Extracting files...


Path to dataset files: /Users/spass/.cache/kagglehub/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images/versions/3


In [14]:
mv /Users/spass/.cache/kagglehub/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images/versions/3 ./v3/

In [8]:
import os
#import cv2
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader

In [16]:
#Dataset class    1 for real, 0 for fake
class RealFakeDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.real_images = self._load_images(os.path.join(root_dir, split, 'REAL'))
        self.fake_images = self._load_images(os.path.join(root_dir, split, 'FAKE'))
        self.all_images = self.real_images + self.fake_images

    def _load_images(self, folder):
        images = []
        for filename in os.listdir(folder):
            if filename.endswith(('.jpg', '.jpeg')):
                images.append(os.path.join(folder, filename))
        return images

    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        img_path = self.all_images[idx]
        image = Image.open(img_path).convert('RGB')
        label = 1 if 'real' in img_path else 0 

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [11]:
#model
import torch.nn as nn
class ViTBinaryClassifier(nn.Module):
    def __init__(self, pretrained=True):
        super(ViTBinaryClassifier, self).__init__()
        self.backbone = timm.create_model("vit_medium_patch16_224", pretrained=pretrained)
        in_features = self.backbone.head.in_features
        self.backbone.head = nn.Identity() 
        self.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1),
            nn.Sigmoid() 
        )
        
    def forward(self, x):
        features = self.backbone(x)
        out = self.classifier(features)
        return out

In [ ]:
def train(model, train_loader, criterion, optimizer, device, epoch, writer):
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.float().to(device).unsqueeze(1)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        total += labels.size(0)
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
    
    avg_loss = running_loss / len(train_loader)
    accuracy = correct / total
    print(f"Epoch {epoch} Train Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
    
    # Logging to TensorBoard and WandB
    writer.add_scalar("Train/Loss", avg_loss, epoch)
    writer.add_scalar("Train/Accuracy", accuracy, epoch)
    wandb.log({"Train Loss": avg_loss, "Train Accuracy": accuracy, "epoch": epoch})
    
    return avg_loss, accuracy

def test(model, test_loader, criterion, device, epoch, writer):
    model.eval()
    running_loss = 0.0
    total = 0
    correct = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.float().to(device).unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_loss = running_loss / len(test_loader)
    accuracy = correct / total
    print(f"Epoch {epoch} Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
    
    writer.add_scalar("Test/Loss", avg_loss, epoch)
    writer.add_scalar("Test/Accuracy", accuracy, epoch)
    wandb.log({"Test Loss": avg_loss, "Test Accuracy": accuracy, "epoch": epoch})
    
    return avg_loss, accuracy

In [18]:
dataset = RealFakeDataset(root_dir='./', split='train', transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

FileNotFoundError: [Errno 2] No such file or directory: './train/REAL'

In [19]:
optimizer

NameError: name 'optimizer' is not defined

In [ ]:
wandb.init(project="vit_variable_resolution", entity="your_wandb_entity")  # Set your wandb entity if needed
writer = SummaryWriter(log_dir="./runs/vit_variable_resolution")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transforms for normalization (the resizing is handled in the dataset)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Using ImageNet stats
                            std=[0.229, 0.224, 0.225])
])

# Create dataset instances
train_dataset = VariableResolutionDataset(root_dir="data/train", resolutions=[224, 256, 288], transform=transform)
test_dataset = VariableResolutionDataset(root_dir="data/test", resolutions=[224, 256, 288], transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Initialize model, criterion, and optimizer
model = ViTBinaryClassifier(pretrained=True).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

num_epochs = 10
for epoch in range(1, num_epochs+1):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device, epoch, writer)
    test_loss, test_acc = test(model, test_loader, criterion, device, epoch, writer)

# Save model checkpoint
torch.save(model.state_dict(), "vit_binary_classifier.pth")
writer.close()
wandb.finish()

In [21]:
import torch.optim as optim
optim.Adam(model.parameters(), lr=3e-4)


NameError: name 'model' is not defined